In [1]:
import pandas as pd
import seaborn as sns
sns.set_theme()
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
# for now naive cleaning
sessions = pd.read_json("data/sessions.jsonl", lines=True).dropna().set_index('timestamp')


## Listen time user/track

In [2]:
listen_events = sessions[sessions['event_type'].isin(['play', 'skip'])]
cache = []
for idx, session in listen_events.groupby('session_id'):
    session = session.reset_index()
    for i in range(len(session) -1):
        curr = session.iloc[i]
        next_v = session.iloc[i+1]
        if curr["track_id"] != next_v["track_id"]:
            continue
        if curr['event_type'] != 'play' or next_v['event_type'] != 'skip':
            continue
        cache.append({
            'timestamp': curr['timestamp'],
            'user_id': curr['user_id'],
            'track_id': curr['track_id'],
            'duration': next_v['timestamp'] - curr['timestamp']
        })
pd.DataFrame(cache)


,timestamp,user_id,track_id,duration
0,2023-01-02 06:17:59.823,101,4JHoOfnad61lljGlVhfFSb,0 days 00:01:31.789000
1,2023-01-02 06:47:13.595,101,44XWtXS1N27SIgdlKwE24B,0 days 00:00:53.529000
2,2023-01-09 14:01:10.164,101,0zMxvCIYqGcn6prnLRjfxW,0 days 00:01:33.767000
3,2023-01-09 14:10:58.131,101,2msZBKdricJsjUMqI4yMbL,0 days 00:03:39.959000
4,2023-01-09 14:21:32.944,101,6WWoWlxdD2OEKY8olGBpW7,0 days 00:03:00.928000
...,...,...,...,...
4357,2023-10-28 21:53:00.023,150,1ZN2Dat0KrO0xqr9t8QP9r,0 days 00:01:42.030000
4358,2023-10-28 21:59:54.293,150,7l1qvxWjxcKpB9PCtBuTbU,0 days 00:01:01.439000
4359,2023-10-28 22:06:14.812,150,6suYoN4gFkanqOBn5yRnJC,0 days 00:00:27.774000
4360,2023-10-28 22:18:04.212,150,0zlnHTHiBQ68ZCAmTw5ozm,0 days 00:03:10.605000


## if skip occurred user/track

In [3]:
sessions[sessions['event_type']=='skip'].groupby(['user_id', 'track_id']).size().reset_index()

,user_id,track_id,0
0,101,04hBFZJ5PeIc8PPe4v1XiA,1
1,101,0803SWqmIJGvZ15B8zsewn,1
2,101,0zMxvCIYqGcn6prnLRjfxW,1
3,101,10Jg3mk0sC54VWlP2opPcA,1
4,101,1CSLeVCXmetBh8IkTPMFdL,1
...,...,...,...
3788,150,77sMIMlNaSURUAXq5coCxE,2
3789,150,7FdUvDkaE24o3FPIWTvzv2,4
3790,150,7FwBtcecmlpc1sLySPXeGE,1
3791,150,7KXjTSCq5nL1LoYtL7XAwS,1


## user session time

In [4]:
sessions.reset_index().groupby(['session_id', 'user_id']).agg(session_start=('timestamp', 'min'), session_end=('timestamp', 'max')).assign(duration=lambda x: x['session_end'] - x['session_start']).reset_index()

,session_id,user_id,session_start,session_end,duration
0,124,101,2023-01-02 05:58:46,2023-01-02 06:48:07.124,0 days 00:49:21.124000
1,125,101,2023-01-09 13:30:57,2023-01-09 14:59:27.143,0 days 01:28:30.143000
2,126,101,2023-01-12 19:26:08,2023-01-12 21:33:19.203,0 days 02:07:11.203000
3,127,101,2023-01-19 21:04:36,2023-01-19 23:43:01.328,0 days 02:38:25.328000
4,128,101,2023-02-01 22:09:05,2023-02-01 23:34:37.991,0 days 01:25:32.991000
...,...,...,...,...,...
643,816,150,2023-09-01 08:44:48,2023-09-01 09:28:18.075,0 days 00:43:30.075000
644,817,150,2023-09-03 16:15:06,2023-09-03 16:35:12.331,0 days 00:20:06.331000
645,818,150,2023-09-23 23:05:26,2023-09-24 00:07:13.194,0 days 01:01:47.194000
646,819,150,2023-10-17 12:01:53,2023-10-17 12:59:34.612,0 days 00:57:41.612000


## skip occurrence with time of day by day

In [20]:
skip_occurrence = sessions[sessions['event_type']=='skip']
skip_occurrence['hour'] = skip_occurrence.index.hour
skip_occurrence['date'] = skip_occurrence.index.date
skip_occurrence.groupby(['date', 'hour']).size().reset_index(name="count")

/tmp/ipykernel_15430/847945583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skip_occurrence['hour'] = skip_occurrence.index.hour
/tmp/ipykernel_15430/847945583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skip_occurrence['date'] = skip_occurrence.index.date


,date,hour,count
0,2023-01-01,15,2
1,2023-01-01,20,1
2,2023-01-03,9,1
3,2023-01-03,22,2
4,2023-01-08,2,5
...,...,...,...
353,2023-10-30,14,5
354,2023-10-30,22,1
355,2023-10-31,18,1
356,2023-11-01,19,3
